### d) Complexidade: Θ(m)
### f) Complexidade Θ(n^2 * m)

In [36]:
from math import inf
from typing import List, Tuple


class Contratos:
  def __init__(self, array: List[List[List[float]]], taxa: float):
    self._taxa = taxa
    self._contratos = array

  @property
  def taxa(self) -> float:
    return self._taxa

  @property
  def num_fornecedores(self):
    return len(self._contratos)
  
  def get_contrato(self, fornecedor: int, periodo_inicial: int, periodo_final: int) -> float:
    return self._contratos[fornecedor][periodo_inicial][periodo_final]

  def get_contratos_periodo(self, periodo_inicial: int, periodo_final: int) -> List[float]:
    contratos = []
    for fornecedor in range(self.num_fornecedores):
      contratos.append(self.get_contrato(fornecedor, periodo_inicial, periodo_final))
    return contracts

  # d)
  def menor_contrato_periodo(self, periodo_inicial: int, periodo_final: int) -> Tuple[int, float]:
    fornecedor: int = len(self._contratos)
    menor_contrato: float = inf
    for f_fornecedor in range(self.num_fornecedores):
      contrato_atual = self.get_contrato(f_fornecedor, periodo_inicial, periodo_final)
      if contrato_atual < menor_contrato:
        fornecedor, menor_contrato = f_fornecedor, contrato_atual
    return fornecedor, menor_contrato

  # f)
  @property
  def menor_contrato(self) -> Tuple[int, int, int, float]:
    fim: int = -1
    inicio: int = -1
    fornecedor: int = -1
    menor_contrato: float = inf

    for i_ini in range(len(self._contratos[0])):
      for j_fim in range(len(self._contratos[0][0])):
        f_forn, menor_contrato_periodo = self.menor_contrato_periodo(i_ini, j_fim)
        if menor_contrato_periodo < menor_contrato:
          fornecedor, inicio, fim, menor_contrato = f_forn, i_ini, j_fim, menor_contrato_periodo
    return fornecedor, inicio, fim, menor_contrato

### a) Leitura do arquivo de entrada em estruturas de dados apropriadas

In [37]:
c: Contratos

with open('entrada.txt', 'r') as arquivo:
  # Lendo a primeira linha do arquivo texto
  n, m, t = arquivo.readline().split()

  t_taxa = float(t) # Valor da taxa de mudança de fornecedor
  n_meses = int(n) # Quantidade de meses
  m_fornecedores = int(m) # Quantidade de fornecedores

  print(f'Período completo em meses (n): {n}\nFornecedores (m): {m}\nTaxa de Mudança (t): {t}')

  # Lendo o restante das linhas do arquivo e gerando a matriz base
  A = [[[inf for k in range(n_meses)] for j in range(n_meses)] for i in range(m_fornecedores)]

  print(f'({len(A)}, {len(A[0])}, {len(A[0][0])})')
  print(f'Tamanho da estrutura de dados: {len(A) * len(A[0]) * len(A[0][0])}')

  for linha in arquivo:
      dados = linha.split(' ');
      i_fornecedor, j_inicio, k_fim = map(lambda i: int(i), dados[:-1])
      A[i_fornecedor - 1][j_inicio - 1][k_fim - 1] = float(dados[-1])
  
  c = Contratos(A, t_taxa)

Período completo em meses (n): 120
Fornecedores (m): 100
Taxa de Mudança (t): 100.0
(100, 120, 120)
Tamanho da estrutura de dados: 1440000


### b) Complexidade da estrutura de dados = Θ(m * n^2)
### c) Criar uma função eficiente que retorna o contrato individual, referente ao período completo de n meses, que possui o menor valor

In [38]:
forn, valor = c.menor_contrato_periodo(0, n_meses - 1)
print(f'O fornecedor {forn + 1} oferece o menor contrato do período completo, que custa: {valor:.2f}')


O fornecedor 61 oferece o menor contrato do período completo, que custa: 687.10


### e) Criar uma função eficiente que retorna o contrato individual de menor valor do mercado, independente do período a que se refere

In [39]:
forn, ini, fim, val = c.menor_contrato
print(f'O menor contrato é fornecido por {forn + 1}, do período de {ini} a {fim} pelo valor de {val:.2f}')

O menor contrato é fornecido por 96, do período de 3 a 3 pelo valor de 10.00
